# Uncertainty analysis with candidate policiy ventilator

In [2]:
# import needed packages
import platypus

In [3]:
import numpy as np
import pandas as pd
import functools
import matplotlib.pyplot as plt
import seaborn as sns

from ema_workbench import (save_results, Constraint, SequentialEvaluator, TimeSeriesOutcome,
                           RealParameter, ScalarOutcome, CategoricalParameter, ema_logging, 
                           perform_experiments, Policy, IntegerParameter)

from ema_workbench.connectors.vensim import VensimModel
from ema_workbench.connectors import vensim
from ema_workbench.em_framework.parameters import Scenario
from ema_workbench.analysis import parcoords

c:\users\pmg00\appdata\local\programs\python\python37\lib\site-packages\ema_workbench\connectors\__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
c:\users\pmg00\appdata\local\programs\python\python37\lib\site-packages\ema_workbench\analysis\prim.py:31: ImportWarning: altair based interactive inspection not available
  "inspection not available"), ImportWarning)


In [4]:
if __name__ == "__main__":
    ema_logging.log_to_stderr(ema_logging.INFO )

In [5]:
#set working directory
wd = 'C:/Users/pmg00/Documents/EPA Master program/EPA Master Thesis/Vensim Models/EMA Workbench connection'

'C:/Users/pmg00/Documents/EPA Master program/EPA Master Thesis/Vensim Models/EMA Workbench connection'

In [6]:
#import vensim model
Model = VensimModel('Model', wd= wd, model_file= './Model/Infection_Model_Testing_02032022_Test.vpmx')

In [7]:
# define uncertainties for ventilators
# exclude uncertainties for PPE 

# ventilator stockpile
Model.uncertainties = [RealParameter('Delivery time of ventilators stockpiling',1, 14),
# RealParameter('Initial ventilators in stockpile',0, 10000),
RealParameter('Preparation time for delivery',1, 10),
RealParameter('Delivery time of ventilators stockpiling',1,21),
                       
#ventilator domestic production
RealParameter('Production time domestic production',1, 5),
RealParameter('Transportation time domestic production',1, 10),
RealParameter('Shipment time domestic production',1, 21),
RealParameter('Delay domestic production setup ventilator',7, 60),
RealParameter('Time to reach maximum production capacity vent dom production',5, 30),
RealParameter('Time to reach maximum procurement capacity vent dom production',5, 30),
RealParameter('Production capacity domestic production ventilator',50, 430),
RealParameter('Raw material domestic production ventilator', 50, 430),

#ventilator loan
RealParameter('Delivery time for available ventilators',1, 21),
# RealParameter('Check up time',1, 21),
RealParameter('Share of ventilators available and fitting',0, 1),
RealParameter('Potentially available ventilators',0, 2400),

# ventilator direct tender
RealParameter('Transportation time direct tender ventilator',1, 10),
RealParameter('Production time ventilators direct tender',1, 10),
RealParameter('Base production capacity direct tender ventilator',50, 500),
RealParameter('Maximum prod direct tender vent',1, 10),
RealParameter('Shipment time direct tender',14, 120),
RealParameter('Share of faulty products',0, 0.5),
RealParameter('Direct tender set up time ventilator',7, 45),

                       
# ventilator innovation
# RealParameter('Urgentness',0, 5),
# RealParameter('Reach ',0, 2000),
# # RealParameter('Government support ',0, 0.1),
# RealParameter('Production time innovation ',1, 7),
# RealParameter('Share of actionable innovations', 0.01, 0.2),
# RealParameter('Transportation time ventilator innovation',1, 10),
# RealParameter('Shipment time innovation ',1, 21),
# RealParameter('Average time to approve and develop products',15, 120),
# RealParameter('Time to reach maximum production capacity vent innovation',10, 90),
# RealParameter('Time to reach maximum procurement capacity vent innovation',10, 90),                     
# RealParameter('Setting up innovation process',5, 60), 
# RealParameter('Base capacity innovation',1, 30),
                       
#ventilator production worldwide
RealParameter('Production time ventilator production worldwide',1, 10),
RealParameter('Base raw material ventilator production worldwide',430, 1720),
RealParameter('Preparation shipment production worldwide',1, 10),
RealParameter('Purchasing power UK as share of GDP per person',0.05 , 0.33),
RealParameter('Shipment time procurement from world market',14, 90),
RealParameter('Maximum increase in procurement capacity vent ww',1, 15),
RealParameter('Time to reach maximum procurement capacity vent worldwide',60, 480),
RealParameter('Maximum days in backlog before increase in procu capacity vent',1, 20),       
RealParameter('Maximum increase in production capacity vent worldwide',1, 15),
RealParameter('Reduction export ventilator',0,1),
RealParameter('Time to reach maximum production capacity vent worldwide',60, 480),
RealParameter('Maximum days in backlog before increase in prod capacity vent',14, 60),   
RealParameter('Maximum days in backlog before increase in procu capacity vent',14, 60),   
RealParameter('Share of vent ready for previous order',0.2, 1),
RealParameter('Maximum transportation time',1, 3),
RealParameter('change in transportation time',7, 60)]
                       
#Decision Framework
# RealParameter('Share of forecasted PPE demand covered by procurement world market',0, 2),


In [8]:
# define decision levers
Model.levers = [IntegerParameter('Switch procurement world market ventilator',0,1),
                IntegerParameter('Switch direct tender ventilators',0,1),
                IntegerParameter('Switch innovation process ventilator',0,1),
                IntegerParameter('Switch loaning ventilators',0,1),
                IntegerParameter('Switch domestic production ventilators',0,1),

RealParameter('Time to check products',1, 5),
RealParameter('Shipment time to hospitals',1, 10),
RealParameter('Order buffer procurement world market vent',0.5, 3),
RealParameter('Order buffer direct tender vent',0.5, 3),
RealParameter('Order buffer domestic production',0.5, 3),
RealParameter('Order buffer innovation',0.5, 3),
RealParameter('Time to establish loaning process',3.5 , 21),
RealParameter('Time horizon for forecast',5 , 30),
RealParameter('Urgentness',0 ,5),
RealParameter('Initial ventilators in stockpile',0, 10000),
RealParameter('Government support ',0, 0.1)]

In [9]:
from LastElement import get_last_element

In [10]:
#import policies from directed search
results_policies = pd.read_csv('./data/4000_candidate_ventilator.csv',delimiter = ';', decimal= ",")
results_policies

,Unnamed: 0,Switch stockpile ventilators,Switch procurement world market ventilator,Switch direct tender ventilators,Switch innovation process ventilator,Switch loaning ventilators,Switch domestic production ventilators,Direct tender set up time ventilator,Check up time,Delay domestic production setup ventilator,...,Order buffer procurement world market vent,Order buffer direct tender vent,Order buffer domestic production,Order buffer innovation,Time to establish loaning process,Time horizon for forecast,Urgentness,Initial ventilators in stockpile,Government support,Coverage ventilator
0,0,0,0,1,0,1,1,7.973928,7.253097,7.263243,...,0.82236,1.399103,0.67371,0.971423,15.268815,21.837371,1.715718,6117.520616,0.079924,13.391387


In [11]:
# drop unnamed and switch stockpile ventilators because they are not needed
results_policies = results_policies.drop(['Unnamed: 0'], axis=1)
results_policies = results_policies.drop(['Switch stockpile ventilators'], axis =1)

#create policies
policies_to_evaluate = []

for i, policy in results_policies.iterrows():
    policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [12]:
policies_to_evaluate

[Policy({'Switch procurement world market ventilator': 0.0, 'Switch direct tender ventilators': 1.0, 'Switch innovation process ventilator': 0.0, 'Switch loaning ventilators': 1.0, 'Switch domestic production ventilators': 1.0, 'Direct tender set up time ventilator': 7.973928126000001, 'Check up time': 7.253096836, 'Delay domestic production setup ventilator': 7.263243202999999, 'Delivery time of ventilators stockpiling': 1.07692105, 'Procurement time ventilators worldwide': 20.83893751, 'Preparation time for delivery': 2.0163410180000003, 'Time to check products': 1.040890702, 'Shipment time to hospitals': 1.002562924, 'Order buffer procurement world market vent': 0.8223601140000001, 'Order buffer direct tender vent': 1.399103152, 'Order buffer domestic production': 0.67370966, 'Order buffer innovation': 0.9714232309999999, 'Time to establish loaning process': 15.26881466, 'Time horizon for forecast': 21.83737059, 'Urgentness': 1.71571834, 'Initial ventilators in stockpile': 6117.5206

In [13]:
#define the outcomes
#policies that perform better in terms of shortage and not 
Model.outcomes = [ScalarOutcome('Coverage ventilator', variable_name='Total normalized coverage ventilator',
                                          kind=ScalarOutcome.MAXIMIZE, function = get_last_element),
                TimeSeriesOutcome("Shortage of ventilators per day"),               
                TimeSeriesOutcome("Normalized shortage of ventilators"),
                TimeSeriesOutcome("Normalized coverage ventilators"),
                TimeSeriesOutcome("Total cost for ventilators"),
                TimeSeriesOutcome("Ventilators supply ready to be shipped")]

In [14]:
# define amount of scenarios
n_scenarios = 1000

#evaluating the policies over the generated scenarios
with SequentialEvaluator(Model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,policies_to_evaluate)

[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 300 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 500 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 700 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 900 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] experiments finished


In [15]:
#save results
save_results(results, './data/results_step3_vent_final.tar.gz')

[MainProcess/INFO] results saved successfully to C:\Users\pmg00\Documents\EPA Master program\EPA Master Thesis\Vensim Models\EMA Workbench connection\data\results_step3_vent_final.tar.gz
